In [8]:
import subprocess
import os
import scipy.io.wavfile as wav
import IPython
import matplotlib.pyplot as plt
import numpy as np
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
%matplotlib inline

ffmpeg_path = '/usr/local/bin/ffmpeg'
wav_path = '/Users/zhouqiangw/Desktop/studio/Highlights/resource/wav'
raw_videos_path = '/Users/zhouqiangw/Desktop/studio/Highlights/resource/raw_video'
highlight_path = '/Users/zhouqiangw/Desktop/studio/Highlights/resource/highlights'

file_list = [f for f in os.listdir(raw_videos_path)]

command = [[ffmpeg_path,'-i', file, wav_path+'/'+os.path.splitext(file)[0]+'.wav'] for file in file_list]

for cmd in command:
    #Remove old wav files
    try:
        os.remove(wav_path+'/'+os.path.splitext(cmd[2])[0]+'.wav')  
    except OSError:
        pass
    process = subprocess.run(cmd, cwd=raw_videos_path)

wav_file_list = [w for w in os.listdir(wav_path) if w.endswith('.mp4')]

### process wav file

In [9]:
def process_wav_files(wav_file_list):
    wav_array = []
    sample_rates = []
    for index, file in enumerate(wav_file_list):
        #Read wav file
        sample_rate, data = wav.read(wav_path+'/'+file)
        #Convert to mono channel
        data = data.astype(np.int32)
        data_mono = (data[:,0] + data[:,1])/2
        wav_duration = len(data)/sample_rate
        sample_rates.append(sample_rate)
        wav_array.append(data_mono)
    return wav_array, sample_rates

In [12]:
def find_energy(moving_average_list):
  energy = []
  for array in moving_average_list:
    energy.append(np.square(array))
  return energy
  
def subsample(array, step):
  ssarray = array[::step].copy()
  return ssarray
  
SAMPLING_FACTOR = 4
WINDOW_LENGTH = 8
sampled_array = []
wav_array, sample_rates = process_wav_files(wav_file_list)

for array in wav_array:
  sampled_array.append(subsample(array, SAMPLING_FACTOR))
  moving_average = find_moving_average(sampled_array, sample_rates, WINDOW_LENGTH, SAMPLING_FACTOR)
  energy = find_energy(moving_average)

ValueError: File format b'\x00\x00\x00 ' not understood. Only 'RIFF' and 'RIFX' supported.

In [ ]:
def find_highlight_times(array, threshold, sample_rate):
  chunks = []
  for i in range(len(array)-1):
    if (array[i]<threshold<array[i+1] or array[i]>threshold>array[i+1]):
      chunks.append((i*SAMPLING_FACTOR/sample_rate))
   return chunks
  
  times = find_highlight_times(energy[1], thresholds[1], sample_rates[1])
  
chunk_list = []
total_highlight_time = 0
for index in range(0, len(times),2):
    difference = times[index+1]-times[index]
    if (difference > 4):
        chunk_list.append([times[index], times[index+1]])
        total_highlight_time = total_highlight_time + difference

In [ ]:
file = open(highlight_path+'/highlight_file.txt','w') 
for index, chunk in enumerate(chunk_list):
    file.write('file '+'\''+highlight_path+'/'+'_highlight_'+str(index)+'.mkv'+'\''+'\n')
    ffmpeg_extract_subclip(raw_videos_path+'/'+file_list[1], int(chunk[0]), int(chunk[1]), targetname=highlight_path+'/'+'_highlight_'+str(index)+'.mkv')
file.close()